In [1]:
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
path_prefix = '/home/joeyresuento/Projects/data_training/data_sets/ml-25m'

In [3]:
movies_df = pd.read_csv(f'{path_prefix}/movies.csv')
ratings_df = pd.read_csv(f'{path_prefix}/ratings.csv')

In [4]:
movies_table = pa.Table.from_pandas(movies_df)
ratings_table = pa.Table.from_pandas(ratings_df)

pq.write_table(movies_table, f'{path_prefix}/movies.parquet')
pq.write_table(ratings_table, f'{path_prefix}/ratings.parquet')

In [5]:
movies = pq.read_table(f'{path_prefix}/movies.parquet')
ratings = pq.read_table(f'{path_prefix}/ratings.parquet')

In [6]:
movies.to_pandas()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
62418,209157,We (2018),Drama
62419,209159,Window of the Soul (2001),Documentary
62420,209163,Bad Poems (2018),Comedy|Drama
62421,209169,A Girl Thing (2001),(no genres listed)


In [7]:
ratings_df0 = ratings.to_pandas()
ratings_df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25000095 entries, 0 to 25000094
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  int64  
dtypes: float64(1), int64(3)
memory usage: 762.9 MB


In [8]:
ratings_df0.head()

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817
2,1,307,5.0,1147868828
3,1,665,5.0,1147878820
4,1,899,3.5,1147868510


In [9]:
overall_mean = ratings_df0['rating'].mean()

In [10]:
# rating_mean = ratings_df0.groupby("movieId").mean()['rating']
# rating_mean.head()

In [11]:
rating_count = ratings_df0.groupby("movieId").count()['rating']
rating_count.head()

movieId
1    57309
2    24228
3    11804
4     2523
5    11714
Name: rating, dtype: int64

In [12]:
ratings_df1 = ratings_df0.groupby('movieId').mean()[['rating']]

In [13]:
ratings_df1['count'] = rating_count
# https://en.wikipedia.org/wiki/IMDb#Rankings
m = 25000
v = ratings_df1['count']
R = ratings_df1['rating']
ratings_df1['weighted_mean'] = ((R * v) + (overall_mean * 25000)) / (v + m)

In [19]:
# ratings_df1.sort_values(by='weighted_mean', ascending=False)

In [15]:
movies_df0 = movies.to_pandas()

In [16]:
# movies_df0[ movies_df0.movieId == 318 ]['title']

314    Shawshank Redemption, The (1994)
Name: title, dtype: object

In [17]:
# movies_df0[ movies_df0.movieId == 858 ]['title']

840    Godfather, The (1972)
Name: title, dtype: object

In [1]:
# not sure if this is efficient
# ratings_df1['title'] = movies_df0[ movies_df0.movieId == ratings_df1 ]['title']